# Notebook 05: Up- and Downstream Wood Ljungdahl Metabolism

This notebook is part of the research paper titled:

Optimizing acetogenic CO(2) utilization through metabolic and process engineering highlights the potential of thermophilic bioproduction

This notebook uses a stoichiometric constraint-based model of the Wood-Ljungdahl pathway and its products to determine the sotichiometry of the energy and redox co-factors needed or produced both up- and downstream of the core metabolite, acetyl-CoA. The model was built using the genome-scale metabolic model of Clostridium ljungdahlii as a template (notebook S05).

Notebook author: Philip J. Gorter de Vries

In [10]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite

In [11]:
WLmodel = cobra.io.read_sbml_model('../Models/WLPathway_sink.xml')

WLmodel.add_boundary(WLmodel.metabolites.get_by_id("accoa_c"), type="sink")

Reaction identifier,SK_accoa_c
Name,Acetyl-CoA sink
Memory address,0x07f8386d2b0f0
Stoichiometry,accoa_c <=> Acetyl-CoA <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


# First forcus on the upstream metabolism, towards acetyl-CoA

Set acetyl-CoA exchange as the objective function and change the medium for carboxydotrophic growth

In [14]:
WLmodel.objective = "SK_accoa_c"

medium_carboxydotrophic = WLmodel.medium
medium_carboxydotrophic['EX_co2_e'] = 0
medium_carboxydotrophic['EX_co_e'] = 2
WLmodel.medium = medium_carboxydotrophic

solution_carboxydotrophic = WLmodel.optimize()
WLmodel.optimize()

,fluxes,reduced_costs
H2td,0.0,-0.0
H2Ot,0.0,-0.0
CO2t,0.0,0.0
COt,-2.0,-0.0
ACt,0.0,0.0
...,...,...
ctfAB_ac,0.0,0.0
ADC,0.0,0.0
ACTt,0.0,0.0
EX_act_e,0.0,0.0


In [16]:
print("Acetyl-CoA: ",WLmodel.slim_optimize())
print("ATP: ",WLmodel.reactions.SK_atp_c.flux)
print("NADH: ",WLmodel.reactions.SK_nadh_c.flux)
print("NADPH: ",WLmodel.reactions.SK_nadph_c.flux)
print("Ferredoxin: ",WLmodel.reactions.SK_fdxo_42_c.flux)
print("H+: ",WLmodel.reactions.SK_h_c.flux)

Builder(model = WLmodel, map_json = '../Models/EscherMaps/WL_Upstream.json', reaction_data = solution_carboxydotrophic.fluxes.to_dict())

Acetyl-CoA:  1.0
ATP:  -1.0
NADH:  -2.0
NADPH:  -1.0
Ferredoxin:  -1.0
H+:  0.0


Builder(reaction_data={'H2td': 0.0, 'H2Ot': 0.0, 'CO2t': 0.0, 'COt': -2.0, 'ACt': 0.0, 'ETOHt': 0.0, 'BTDt_RR'…

In [17]:
medium_homoacetegonic = WLmodel.medium
medium_homoacetegonic['EX_co2_e'] = 2
medium_homoacetegonic['EX_co_e'] = 0
WLmodel.medium = medium_homoacetegonic

solution_homoacetegonic = WLmodel.optimize()
WLmodel.optimize()

,fluxes,reduced_costs
H2td,0.0,-0.0
H2Ot,-2.0,-0.0
CO2t,2.0,0.0
COt,0.0,-0.0
ACt,0.0,0.0
...,...,...
ctfAB_ac,0.0,0.0
ADC,0.0,0.0
ACTt,0.0,0.0
EX_act_e,0.0,0.0


In [19]:
print("Acetyl-CoA: ",WLmodel.slim_optimize())
print("ATP: ",WLmodel.reactions.SK_atp_c.flux)
print("NADH: ",WLmodel.reactions.SK_nadh_c.flux)
print("NADPH: ",WLmodel.reactions.SK_nadph_c.flux)
print("Ferredoxin: ",WLmodel.reactions.SK_fdxo_42_c.flux)
print("H+: ",WLmodel.reactions.SK_h_c.flux)

Builder(model = WLmodel, map_json = '../Models/EscherMaps/WL_Upstream.json', reaction_data = solution_homoacetegonic.fluxes.to_dict())

Acetyl-CoA:  1.0
ATP:  -1.0
NADH:  -2.0
NADPH:  -1.0
Ferredoxin:  1.0
H+:  -4.0


Builder(reaction_data={'H2td': 0.0, 'H2Ot': -2.0, 'CO2t': 2.0, 'COt': 0.0, 'ACt': 0.0, 'ETOHt': 0.0, 'BTDt_RR'…

# Now forcus on the downstream metabolism, product formation from acetyl-CoA

Extract data manually

In [20]:
medium = WLmodel.medium

medium['EX_co2_e'] = 1000
medium['EX_h2_e'] = 10
medium['EX_co_e'] = 0

WLmodel.medium = medium

In [21]:
WLmodel.reactions.CODH_ACS.bounds = 0,0 # cut off the upstream pathway
WLmodel.reactions.SK_accoa_c.bounds = -1,1

WLmodel.reactions.SK_accoa_c

Reaction identifier,SK_accoa_c
Name,Acetyl-CoA sink
Memory address,0x07f8386d2b0f0
Stoichiometry,accoa_c <=> Acetyl-CoA <=>
GPR,
Lower bound,-1
Upper bound,1


In [22]:
objectives = ["EX_ac_e","EX_etoh_e","EX_act_e","EX_but_e","EX_btoh_e","EX_btd_RR_e"]

for x in objectives:
    WLmodel.objective = x
    WLmodel.optimize()
    print(WLmodel.reactions.get_by_id(x).name)
    print("  Product: ",WLmodel.slim_optimize())
    print("  ATP: ",WLmodel.reactions.SK_atp_c.flux)
    print("  NADH: ",WLmodel.reactions.SK_nadh_c.flux)
    print("  NADPH: ",WLmodel.reactions.SK_nadph_c.flux)
    print("  Ferredoxin: ",WLmodel.reactions.SK_fdxo_42_c.flux)
    print("  H+: ",WLmodel.reactions.SK_h_c.flux,"\n")

Acetate exchange
  Product:  1.0
  ATP:  1.0
  NADH:  0.0
  NADPH:  0.0
  Ferredoxin:  0.0
  H+:  0.0 

Ethanol exchange
  Product:  1.0
  ATP:  1.0
  NADH:  -1.0
  NADPH:  0.0
  Ferredoxin:  1.0
  H+:  -4.0 

Acetone exchange
  Product:  0.5
  ATP:  0.5
  NADH:  0.0
  NADPH:  0.0
  Ferredoxin:  0.0
  H+:  -0.5 

Butyrate (n-C4:0) exchange
  Product:  0.5
  ATP:  0.5
  NADH:  -1.5
  NADPH:  0.0
  Ferredoxin:  -0.5
  H+:  -0.5 

Butanol exchange
  Product:  0.5
  ATP:  0.0
  NADH:  -2.5
  NADPH:  0.0
  Ferredoxin:  -0.5
  H+:  -1.5 

 R R  2 3 Butanediol C4H10O2 exchange
  Product:  0.5
  ATP:  0.0
  NADH:  -0.5
  NADPH:  0.0
  Ferredoxin:  1.0
  H+:  -2.5 



## Closer look, change x within [0:5] to view different product

In [24]:
x = 1

WLmodel.objective = objectives[x]

solution = WLmodel.optimize()

Builder(model = WLmodel, map_json = '../Models/EscherMaps/WL_Downstream.json', reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'H2td': 0.0, 'H2Ot': -1.0, 'CO2t': 0.0, 'COt': 0.0, 'ACt': 0.0, 'ETOHt': -1.0, 'BTDt_RR…